In [ ]:
%%writefile requirements.txt

openai==1.42.0
azure-identity==1.17.1

In [ ]:
!python -m ensurepip --upgrade --user

%pip install -r requirements.txt --user

In [8]:
tenant_id = ""
client_id = ""
client_secret = ""
client_id_scope = f"api://{client_id}/user_impersonation"
redirect_uri = "http://localhost:8000"
apim_name = ""
apim_subscription_key = ""
openai_client_id = ""
openai_id_scope = f"api://{openai_client_id}/user_impersonation"
openai_api_version = "2024-06-01"
chat_deployment_name = "gpt-4o-2024-05-13"
embedding_deployment_name = "text-embedding-3-large-1"

In [ ]:
from openai import AzureOpenAI
from azure.identity import DefaultAzureCredential, OnBehalfOfCredential, InteractiveBrowserCredential, get_bearer_token_provider

#this example uses an app registration with the "Mobile and desktop applications" redirect URI set to "http://localhost:8000"
credential = InteractiveBrowserCredential(
   tenant_id=tenant_id,
   client_id=client_id,
   redirect_uri=redirect_uri)

token_provider = get_bearer_token_provider(credential, client_id_scope)

obo_credential = OnBehalfOfCredential(
    tenant_id=tenant_id,
    client_id=client_id,
    client_secret=client_secret,
    user_assertion=token_provider(),
)

obo_token_provider = get_bearer_token_provider(obo_credential, openai_id_scope)

client = AzureOpenAI(
    api_version=openai_api_version,
    azure_endpoint="https://" + apim_name + ".azure-api.net",
    azure_ad_token_provider=obo_token_provider,
    default_headers={"Ocp-Apim-Subscription-Key": apim_subscription_key}
)

Call OpenAI through APIM

In [ ]:
response = client.chat.completions.create(
    model=chat_deployment_name,
    messages=[
        {
            "role": "user",
            "content": "Hello!"
        }
    ]
)

print(response.to_json())

Call OpenAI through APIM in a loop

In [ ]:
for i in range(1000):
    client.chat.completions.create(
        model=chat_deployment_name,
        messages=[
            {
                "role": "system", "content": """Assistant is an intelligent chatbot designed to help users answer technical questions about Azure OpenAI Serivce. Only answer questions using the context below and if you're not sure of an answer, you can say 'I don't know'.
                Context:
                - Azure OpenAI Service provides REST API access to OpenAI's powerful language models including the GPT-3, Codex and Embeddings model series.
                - Azure OpenAI Service gives customers advanced language AI with OpenAI GPT-3, Codex, and DALL-E models with the security and enterprise promise of Azure. Azure OpenAI co-develops the APIs with OpenAI, ensuring compatibility and a smooth transition from one to the other.
                - At Microsoft, we're committed to the advancement of AI driven by principles that put people first. Microsoft has made significant investments to help guard against abuse and unintended harm, which includes requiring applicants to show well-defined use cases, incorporating Microsoft’s principles for responsible AI use."""
            },
            {
                "role": "user", 
                "content": "What is Azure OpenAI Service?"}
        ]
    )